# Machine Learning Portfolio 3

|Name|Github|Kaggle|
|----|------|------|
|Henry Lau|HenryLau08|Henry Lau|
|Mohamed Belaachir|mobelaachir|Mo Belaachir|
|Jayden Debi-Tewari|Jaydendt1|jaydendt123|
|Quincy Soerohardjo|quincysoerohardjo2002|Quincy Soerohardjo|
|Mattias Aareleid|mattyonaize|Mattias Aareleid|

## Table of Contents
- [Data Overview](#data-overview)
- [Exploratory Data Analysis](#exploratory-data-analysis)
- [Feature Engineering](#feature-engineering)
- [Modeling](#modeling)
    
- [Results](#results)
    - [Overview](#overview)
    - [Scores](#scores)
- [Conclusion & Advice](#conclusion--advice)
- [Sources](#sources)

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score



from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)

## Data Overview

In [ ]:
labels_new = pd.read_csv('labels_new.csv')
sample_submission = pd.read_csv('sample_submission.csv')

print("labeled data")
display(labels_new.head(10))
print()
print("sample submission voor kaggle")
display(sample_submission.head(10))

labeled data


,filename,genre
0,m00248.wav,metal
1,m00230.wav,country
2,m00637.wav,hiphop
3,m00627.wav,metal
4,m00138.wav,reggae
5,m00192.wav,classical
6,m00429.wav,hiphop
7,m00623.wav,reggae
8,m00002.wav,jazz
9,m00039.wav,reggae



sample submission voor kaggle


,filename,genre
0,metal.00032.wav,classical
1,pop.00023.wav,blues
2,classical.00076.wav,blues
3,classical.00021.wav,rock
4,metal.00052.wav,classical
5,classical.00040.wav,reggae
6,pop.00097.wav,hiphop
7,classical.00005.wav,pop
8,classical.00056.wav,pop
9,metal.00073.wav,classical


## Exploratory Data Analysis

# t-Distributed Stochastic Neighbor Embedding (t-SNE)

t-Distributed Stochastic Neighbor Embedding (t-SNE) is een statistische techniek die wordt gebruikt om hoog-dimensionale data te visualiseren door deze om te zetten naar een lagere dimensie, vaak 2D of 3D. Het doel is om de structuur van de data in de lagere dimensie te behouden, zodat vergelijkbare datapunten dicht bij elkaar komen te liggen en verschillende datapunten verder uit elkaar.

## Werking van het t-SNE-algoritme

Het t-SNE-algoritme bestaat uit twee hoofdcomponenten:

1. **Constructie van de waarschijnlijkheidsverdeling in de hoog-dimensionale ruimte:**
   - Voor elk paar van hoog-dimensionale objecten wordt een waarschijnlijkheid berekend die de gelijkenis tussen deze objecten weerspiegelt.
   - De gelijkenis wordt gemodelleerd met behulp van een Gaussische verdeling, waarbij punten die dichter bij elkaar liggen een hogere waarschijnlijkheid krijgen en punten die verder uit elkaar liggen een lagere waarschijnlijkheid.

2. **Constructie van de waarschijnlijkheidsverdeling in de laag-dimensionale ruimte en minimalisatie van de Kullback-Leibler-divergentie:**
   - In de laag-dimensionale ruimte wordt een vergelijkbare waarschijnlijkheidsverdeling gedefinieerd, maar hier wordt de waarschijnlijkheid tussen punten berekend met behulp van een Student's t-verdeling met één vrijheidsgraad.
   - Het algoritme minimaliseert vervolgens de Kullback-Leibler-divergentie (KL-divergentie) tussen de waarschijnlijkheidsverdelingen van de hoog-dimensionale ruimte en de laag-dimensionale ruimte door de posities van de punten in de laag-dimensionale ruimte aan te passen via gradient descent optimalisatie.

## Wiskundige Formulering

### Stap 1: Berekening van de waarschijnlijkheidsverdeling in de hoog-dimensionale ruimte

Voor een dataset met \( N \) hoog-dimensionale objecten \( \mathbf{x}_1, \dots, \mathbf{x}_N \), wordt de voorwaardelijke waarschijnlijkheid \( p_{j|i} \) berekend voor elk paar punten \( \mathbf{x}_i \) en \( \mathbf{x}_j \) (waarbij \( i \neq j \)):

$$
p_{j|i} = \frac{\exp\left(-\frac{\lVert\mathbf{x}_i - \mathbf{x}_j\rVert^2}{2\sigma_i^2}\right)}{\sum_{k \neq i} \exp\left(-\frac{\lVert\mathbf{x}_i - \mathbf{x}_k\rVert^2}{2\sigma_i^2}\right)}
$$

Waarbij:
- \( \mathbf{x}_i \) en \( \mathbf{x}_j \) de datapunten zijn in de hoog-dimensionale ruimte.
- \( \lVert \mathbf{x}_i - \mathbf{x}_j \rVert^2 \) de kwadratische Euclidische afstand is tussen de punten \( \mathbf{x}_i \) en \( \mathbf{x}_j \).
- \( \sigma_i \) is de bandbreedte van de Gaussische kernel voor punt \( \mathbf{x}_i \).
- De noemer normaliseert de waarschijnlijkheden zodat ze optellen tot 1 voor elk punt.

De symmetrische waarschijnlijkheid \( p_{ij} \) wordt berekend door de voorwaardelijke waarschijnlijkheden \( p_{j|i} \) en \( p_{i|j} \) te combineren:

$$
p_{ij} = \frac{p_{j|i} + p_{i|j}}{2N}
$$

Waarbij:
- \( p_{ij} \) de symmetrische waarschijnlijkheid is dat \( \mathbf{x}_i \) en \( \mathbf{x}_j \) dicht bij elkaar liggen.
- \( N \) is het totaal aantal datapunten in de dataset.

### Stap 2: Berekening van de waarschijnlijkheidsverdeling in de laag-dimensionale ruimte

In de laag-dimensionale ruimte (meestal 2D of 3D) worden de waarschijnlijkheden \( q_{ij} \) berekend met behulp van een Student's t-verdeling met één vrijheidsgraad (wat gelijk is aan de Cauchy-verdeling). De waarschijnlijkheid tussen twee punten \( \mathbf{y}_i \) en \( \mathbf{y}_j \) wordt als volgt gedefinieerd:

$$
q_{ij} = \frac{(1 + \lVert \mathbf{y}_i - \mathbf{y}_j \rVert^2)^{-1}}{\sum_{k} \sum_{l \neq k} (1 + \lVert \mathbf{y}_k - \mathbf{y}_l \rVert^2)^{-1}}
$$

Waarbij:
- \( \mathbf{y}_i \) en \( \mathbf{y}_j \) de coördinaten van de punten zijn in de laag-dimensionale ruimte.
- \( \lVert \mathbf{y}_i - \mathbf{y}_j \rVert^2 \) de kwadratische Euclidische afstand is tussen de laag-dimensionale punten \( \mathbf{y}_i \) en \( \mathbf{y}_j \).
- De noemer normaliseert de waarschijnlijkheden zodat ze optellen tot 1 voor de laag-dimensionale ruimte.

### Stap 3: Minimalisatie van de Kullback-Leibler-divergentie

Het doel van t-SNE is om de Kullback-Leibler-divergentie tussen de twee waarschijnlijkheidsverdelingen \( P \) en \( Q \) (de verdelingen in respectievelijk de hoog-dimensionale en laag-dimensionale ruimte) te minimaliseren. Dit wordt gedaan door de volgende formule:

$$
\mathrm{KL}(P \parallel Q) = \sum_{i \neq j} p_{ij} \log \frac{p_{ij}}{q_{ij}}
$$

Waarbij:
- \( \mathrm{KL}(P \parallel Q) \) de Kullback-Leibler-divergentie is, een maat voor het verschil tussen de verdelingen \( P \) en \( Q \).
- \( p_{ij} \) is de waarschijnlijkheid uit de hoog-dimensionale ruimte (berekend in stap 1).
- \( q_{ij} \) is de waarschijnlijkheid uit de laag-dimensionale ruimte (berekend in stap 2).

Deze KL-divergentie wordt geminimaliseerd door het optimaliseren van de posities van de punten in de laag-dimensionale ruimte via gradient descent.

## Bronnen
 
 t-distributed stochastic neighbor embedding. (2024, 12 januari). Wikipedia. [Link naar Wikipedia](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)
 
 ML | t-Distributed Stochastic Neighbor Embedding (t-SNE) Algorithm. (z.d.). GeeksforGeeks. [Link naar GeeksforGeeks](https://www.geeksforgeeks.org/ml-t-distributed-stochastic-neighbor-embedding-t-sne-algorithm/)




---

# Principal Component Analysis (PCA)

## Wiskundige werking van PCA

PCA is een lineaire techniek voor dimensiereductie die gebruikmaakt van een wiskundige transformatie om een dataset met hoge dimensies te projecteren op een lager-dimensionale ruimte. Het doel is om zoveel mogelijk variatie (informatie) in de data te behouden. Hieronder worden de stappen van PCA uitgelegd.

---

### **1. Standaardisatie van de data**

De dataset wordt eerst geschaald zodat elke feature een gemiddelde van 0 en een standaardafwijking van 1 heeft. Dit gebeurt met de volgende formule:

$$
x' = \frac{x - \bar{x}}{\sigma}
$$

### Waarbij:
- \( x \): De waarde van een specifieke feature in de dataset (een datawaarde).
- \( \bar{x} \): Het gemiddelde van alle waarden van die specifieke feature.
- \( \sigma \): De standaardafwijking van die specifieke feature.
- \( x' \): De gestandaardiseerde waarde (met gemiddelde 0 en standaardafwijking 1).

---

### **2. Berekening van de covariantiematrix**

De covariantiematrix meet de variabiliteit tussen de verschillende features. Deze wordt berekend met de formule:

$$
\Sigma = \frac{1}{n-1} X^T X
$$

### Waarbij:
- \( \Sigma \): De **covariantiematrix**, een vierkante matrix waarin elke cel de covariantie tussen twee features \( x_i \) en \( x_j \) weergeeft.
- \( X \): De data **matrix** na standaardisatie. Elke rij vertegenwoordigt een datapuntenvector, en elke kolom stelt een gestandaardiseerde feature voor.
- \( X^T \): De getransponeerde matrix van \( X \), waarin de rijen en kolommen zijn omgewisseld.
- \( n \): Het aantal datapunten in de dataset.
- \( n-1 \): Correctiefactor die rekening houdt met het aantal vrijheidsgraden bij het berekenen van de covariantie.

---

### **3. Eigenwaarden en eigenvectoren**

Voor de covariantiematrix \( \Sigma \) worden de eigenwaarden en eigenvectoren berekend. Deze voldoen aan de volgende eigenschap:

$$
\Sigma v = \lambda v
$$

### Waarbij:
- \( \lambda \): Een **eigenwaarde**, die de hoeveelheid variatie aangeeft die door de corresponderende eigenvector wordt uitgelegd.
- \( v \): Een **eigenvector**, die de richting van maximale variatie in de data weergeeft.
- \( \Sigma \): De covariantiematrix.

De eigenwaarden en eigenvectoren worden gebruikt om de belangrijkste componenten te bepalen.

---

### **4. Selectie van belangrijkste componenten**

De eigenvectoren worden gerangschikt op basis van hun bijbehorende eigenwaarden \( \lambda \). Alleen de top \( k \) eigenvectoren met de hoogste eigenwaarden worden geselecteerd. Dit bepaalt de nieuwe dimensie \( k \).

- \( k \): Het aantal dimensies dat overblijft na dimensiereductie. Dit wordt gekozen door de gebruiker, afhankelijk van hoeveel variatie behouden moet blijven.

---

### **5. Transformatie naar de lagere dimensie**

Na selectie van de belangrijkste eigenvectoren wordt de originele data getransformeerd naar de lagere dimensie. De transformatie is als volgt:

$$
Z = X W
$$

### Waarbij:
- \( Z \): De getransformeerde data in de lagere-dimensionale ruimte. Elke rij is een datapuntenvector in deze nieuwe \( k \)-dimensionale ruimte.
- \( X \): De gestandaardiseerde data matrix.
- \( W \): De matrix bestaande uit de geselecteerde \( k \) eigenvectoren (elk als een kolom).

Hierbij projecteert \( W \) de originele data \( X \) op de ruimte die wordt gedefinieerd door de \( k \) belangrijkste eigenvectoren.


[(GeeksforGeeks, 2024)](https://www.geeksforgeeks.org/principal-component-analysis-pca/)
[(Wikipedia, 2024)](https://en.wikipedia.org/wiki/Principal_component_analysis)

# Bronnen

Wikipedia contributors. (2024, 12 december). Principal Component Analysis. Wikipedia. https://en.wikipedia.org/wiki/Principal_component_analysis

GeeksforGeeks. (2024, 17 september). Principal Component Analysis (PCA). GeeksforGeeks. https://www.geeksforgeeks.org/principal-component-analysis-pca/



# Non-negatieve Matrixfactorisatie (NMF)

## Wiskundige werking van NMF

Non-negatieve Matrixfactorisatie (NMF) is een techniek voor dimensiereductie die een niet-negatieve matrix \( V \) ontleedt in twee niet-negatieve matrices \( W \) en \( H \). Het doel is om de oorspronkelijke data te benaderen door de productmatrix \( WH \), waarbij alle elementen niet-negatief zijn. Dit is vooral nuttig in toepassingen waar de gegevens van nature niet-negatief zijn, zoals bij afbeeldingen, audio en tekstgegevens. 

---

### **1. Factorisatie van de matrix**

NMF probeert de matrix \( V \) te factoriseren in twee matrices \( W \) en \( H \) zodanig dat:

$$
V \approx WH
$$

**Betekenis van de symbolen:**
- \( V \): De oorspronkelijke niet-negatieve data matrix van dimensie \( m \times n \).
- \( W \): Een niet-negatieve matrix van dimensie \( m \times k \), waarbij elke kolom een basisvector vertegenwoordigt.
- \( H \): Een niet-negatieve matrix van dimensie \( k \times n \), waarbij elke rij de coëfficiënten bevat voor de lineaire combinatie van basisvectoren om de kolommen van \( V \) te benaderen.
- \( k \): Het aantal latente componenten of basisvectoren, gekozen op basis van de gewenste reductie en interpretatie.

---

### **2. Optimalisatieprobleem**

Het doel is om \( W \) en \( H \) te vinden die de reconstructiefout minimaliseren. Een veelgebruikte maat voor deze fout is de Frobeniusnorm van het verschil tussen \( V \) en \( WH \):

$$
\min_{W, H} \| V - WH \|_F^2
$$

**Onder de voorwaarden:**
- \( W \geq 0 \)
- \( H \geq 0 \)

Hierbij zorgt de niet-negativiteitsbeperking ervoor dat de resulterende matrices \( W \) en \( H \) geen negatieve elementen bevatten, wat bijdraagt aan de interpretatie van de componenten. 

---

### **3. Iteratieve update-algoritmen**

Er zijn verschillende algoritmen ontwikkeld om \( W \) en \( H \) iteratief bij te werken om de reconstructiefout te minimaliseren. Een bekende benadering is het gebruik van multiplicatieve update-regels,  [(Lee,D,D & Seung, 2001)](https://proceedings.neurips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf
)

De update-regels zijn als volgt:

$$
H_{ij} \leftarrow H_{ij} \frac{(W^T V)_{ij}}{(W^T W H)_{ij}}
$$

$$
W_{ij} \leftarrow W_{ij} \frac{(V H^T)_{ij}}{(W H H^T)_{ij}}
$$

Deze regels worden herhaald totdat de reconstructiefout convergeert naar een minimumwaarde. 


[(Wikipedia,2024)](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization)
[(GeeksforGeeks, 2024)](https://www.geeksforgeeks.org/introduction-to-non-negative-matrix-factorization/)

# Bronnen
Wikipedia contributors. (2024, 12 december). Non-negative Matrix Factorization. Wikipedia. https://en.wikipedia.org/wiki/Non-negative_matrix_factorization

Lee, D. D., & Seung, H. S. (2001). Algorithms for Non-negative Matrix Factorization. Advances in Neural Information Processing Systems, 13, 556–562. https://proceedings.neurips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf

\GeeksforGeeks. (2024, 17 september). Introduction to Non-negative Matrix Factorization. GeeksforGeeks. https://www.geeksforgeeks.org/introduction-to-non-negative-matrix-factorization/


## Feature Engineering

\Sigma

## Modeling

## Results

## Conclusion & Advice

## Sources